## Term Frequency - Inverse Document Frequency

Bag-of-words comparisons are not very good when all tokens are treated the same: some tokens are more important than others. Weights give us a way to specify which tokens to favor. With weights, when we compare documents, instead of counting common tokens, we sum up the weights of common tokens. A good heuristic for assigning weights is called "Term-Frequency/Inverse-Document-Frequency," or tf-idf for short.

We will build and test our TF-IDF model using the data from the Quora challenge on Kaggle (https://www.kaggle.com/c/quora-question-pairs) were the goal was to identify the duplicate questions

In [28]:
import pandas as pd
import re

### TF

TF rewards tokens that appear many times in the same document. It is computed as the frequency of a token in a document, that is, if document *d* contains 100 tokens and token *t* appears in *d* 5 times, then the TF weight of *t* in *d* is *5/100 = 1/20*. The intuition for TF is that if a word occurs often in a document, then it is more important to the meaning of the document.

### IDF

IDF rewards tokens that are rare overall in a dataset. The intuition is that it is more significant if two documents share a rare word than a common one. IDF weight for a token, *t*, in a set of documents, *U*, is computed as follows:
* Let *N* be the total number of documents in *U*
* Find *n(t)*, the number of documents in *U* that contain *t*
* Then *IDF(t) = N/n(t)*.

Note that *n(t)/N* is the frequency of *t* in *U*, and *N/n(t)* is the inverse frequency.

> **Note on terminology**: Sometimes token weights depend on the document the token belongs to, that is, the same token may have a different weight when it's found in different documents.  We call these weights *local* weights.  TF is an example of a local weight, because it depends on the length of the source.  On the other hand, some token weights only depend on the token, and are the same everywhere that token is found.  We call these weights *global*, and IDF is one such weight.

### TF-IDF

Finally, to bring it all together, the total TF-IDF weight for a token in a document is the product of its TF and IDF weights.
[tfidf]: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [29]:
# Import dataset and stop words
dataset = pd.read_csv("input/train_quora.csv", sep = ',')
dataset = dataset[:1000] # just a sample in order to speed up the computation

### Stop Words Dictionary ###
f = open("input/english-stop-words-large.txt",'r')
stop_words_raw = f.readlines()
stop_words = [i.replace('\n',"") for i in stop_words_raw]

split_regex = r'\W+'

In [31]:
def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    x = re.split(split_regex,string.lower())
    return [i for i in x if i != '']

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    x = simpleTokenize(string)
    return [i for i in x if i not in stop_words]

In [32]:
# Tokenize the dataset columnwise
dataset["question1_token"] = dataset.apply(lambda row : tokenize(row["question1"]),axis = 1)
dataset["question2_token"] = dataset.apply(lambda row : tokenize(row["question2"]),axis = 1)

Term Frequency implementation

In [33]:
def countTokens(df,q):
    """ Count and return the number of tokens
    Args:
        dataframe, question column name
    Returns:
        count: count of all tokens
    """
    return df.apply(lambda row: len(row[q]), axis = 1).sum()

def tf(tokens):
    """ Compute TF
    Args:
        tokens (list of str): input list of tokens from tokenize
    Returns:
        dictionary: a dictionary of tokens to its TF values
    """
    dictionary = {}
    for i in tokens:
      dictionary[i] = tokens.count(i)/float(len(tokens))
    return dictionary

# Let's see the result on a famous sentence often employed in text analytics
print tf(tokenize("The quick brown fox jumps over the lazy dog"))

{'brown': 0.16666666666666666, 'lazy': 0.16666666666666666, 'jumps': 0.16666666666666666, 'fox': 0.16666666666666666, 'dog': 0.16666666666666666, 'quick': 0.16666666666666666}


Build the corpus with all the questions

In [34]:
corpus1 = pd.DataFrame()
corpus2 = pd.DataFrame()
corpus1["id"] = dataset[['qid1']]
corpus1["tf"] = dataset[['qid1',"question1_token"]].apply(lambda row: tf(row["question1_token"]), axis = 1)
corpus2["id"] = dataset[['qid2']]
corpus2["tf"] = dataset[['qid2',"question2_token"]].apply(lambda row: tf(row["question2_token"]), axis = 1)
corpus = pd.concat([corpus1,corpus2])

IDF implementation

In [35]:
def idfs(corpus):
    """ Compute IDF
    Args:
        corpus dataframe
    Returns:
        dictionary of {tokens:idf}
    """

    N = len(corpus)
    tokens = [item for sublist in list(corpus.apply(lambda row: row["tf"].keys(),axis = 1)) for item in sublist]
    unique_tokens = list(set(tokens))
    idf_dict = {}
    for tok in unique_tokens:
        count = 0
        for doc in corpus.tf:
            if tok in doc.keys():
                count += 1
                continue
        idf_dict[tok] = N/float(count)

    return idf_dict

idf_glob = idfs(corpus)

TF-IDF implementation

In [36]:
def tfidf(tokens, idfs):
    """ Compute TF-IDF
    Args:
        tokens (list of str): input list of tokens from tokenize
        idfs (dictionary): record to IDF value
    Returns:
        dictionary: a dictionary of records to TF-IDF values
    """
    tfs = tf(tokens)
    tfIdfDict = {k: v*idfs[k] for (k,v) in tfs.items()}
    return tfIdfDict

Cosine Similarity
Now we are ready to do text comparisons in a formal way. The metric of string distance we will use is called **[cosine similarity][cosine]**. We will treat each document as a vector in some high dimensional space. Then, to compare two documents we compute the cosine of the angle between their two document vectors. This is *much* easier than it sounds.

The first question to answer is how do we represent documents as vectors? The answer is familiar: bag-of-words! We treat each unique token as a dimension, and treat token weights as magnitudes in their respective token dimensions. For example, suppose we use simple counts as weights, and we want to interpret the string "Hello, world!  Goodbye, world!" as a vector. Then in the "hello" and "goodbye" dimensions the vector has value 1, in the "world" dimension it has value 2, and it is zero in all other dimensions.

The next question is: given two vectors how do we find the cosine of the angle between them? Recall the formula for the dot product of two vectors:
\\[ a \cdot b = \| a \| \| b \| \cos \theta \\]
Here \\( a \cdot b = \sum a_i b_i \\) is the ordinary dot product of two vectors, and \\( \|a\| = \sqrt{ \sum a_i^2 } \\) is the norm of \\( a \\).

We can rearrange terms and solve for the cosine to find it is simply the normalized dot product of the vectors. With our vector model, the dot product and norm computations are simple functions of the bag-of-words document representations, so we now have a formal way to compute similarity:
\\[ similarity = \cos \theta = \frac{a \cdot b}{\|a\| \|b\|} = \frac{\sum a_i b_i}{\sqrt{\sum a_i^2} \sqrt{\sum b_i^2}} \\]

Setting aside the algebra, the geometric interpretation is more intuitive. The angle between two document vectors is small if they share many tokens in common, because they are pointing in roughly the same direction. For that case, the cosine of the angle will be large. Otherwise, if the angle is large (and they have few words in common), the cosine is small. Therefore, cosine similarity scales proportionally with our intuitive sense of similarity.
[cosine]: https://en.wikipedia.org/wiki/Cosine_similarity

In [37]:
import math


##### COSINE SIMILARITY ####

def dotprod(a, b):
    """ Compute dot product
    Args:
        a (dictionary): first dictionary of record to value
        b (dictionary): second dictionary of record to value
    Returns:
        dotProd: result of the dot product with the two input dictionaries
    """
    common_key = list(set(a.keys()).intersection(set(b.keys())))
    return sum([a[i]*b[i] for i in common_key])

def norm(a):
    """ Compute square root of the dot product
    Args:
        a (dictionary): a dictionary of record to value
    Returns:
        norm (float): the square root of the dot product value
    """
    return math.sqrt(sum([i**2 for i in a.values()]))

def cossim(a, b):
    """ Compute cosine similarity
    Args:
        a (dictionary): first dictionary of record to value
        b (dictionary): second dictionary of record to value
    Returns:
        cossim: dot product of two dictionaries divided by the norm of the first dictionary and
                then by the norm of the second dictionary
    """
    return dotprod(a, b)/(norm(a)*norm(b))

testVec1 = {'foo': 2, 'bar': 3, 'baz': 5 }
testVec2 = {'foo': 1, 'bar': 0, 'baz': 20 }
dp = dotprod(testVec1, testVec2)
nm = norm(testVec1)
cs = cossim(testVec1, testVec2)
print dp, nm, cs

102 6.16441400297 0.826297021229


In [38]:
def cosineSimilarity(string1, string2, idfsDictionary):
    """ Compute cosine similarity between two strings
    Args:
        string1 (str): first string
        string2 (str): second string
        idfsDictionary (dictionary): a dictionary of IDF values
    Returns:
        cossim: cosine similarity value
    """
    w1 = tfidf(tokenize(string1),idfsDictionary)
    w2 = tfidf(tokenize(string2),idfsDictionary)
    return cossim(w1, w2)

In [39]:
dataset["flag_len"] = dataset.apply(lambda row: 1 if (len(row["question1_token"])==0 or  len(row["question2_token"])==0) else 0, axis = 1) 

dataset = dataset[dataset.flag_len == 0]

dataset["cos_sim"] = dataset.apply(lambda row: cosineSimilarity(row["question1"], row["question2"], idf_glob), axis = 1)

# Similarity == Duplicate?
As you may notice in the two tables printed below, some additional tuning is required to better understand when two questions are a duplicate, mainly (a) fine tune the stop words, as in the provided dictionary there are probably too many, and (b) build a synonyms table

In [53]:
dataset.sort_values(by='cos_sim', ascending=False)[0:10]

,id,qid1,qid2,question1,question2,is_duplicate,question1_token,question2_token,flag_len,cos_sim
253,253,507,508,What are the qualities of a good leader?,What are some good qualities of a leader?,1,"[qualities, good, leader]","[good, qualities, leader]",0,1.0
568,568,1134,1135,How do I start writing again?,How do I start writing?,0,"[start, writing]","[start, writing]",0,1.0
153,153,307,308,At what age should someone lose their virginity?,"At what age, how, and where did you lose your ...",0,"[age, lose, virginity]","[age, lose, virginity]",0,1.0
922,922,1839,1840,How should I start meditating?,"How should I start meditating, and when?",1,"[start, meditating]","[start, meditating]",0,1.0
222,222,445,446,How can I find job in Japan?,How can I find an IT job in Japan?,0,"[find, job, japan]","[find, job, japan]",0,1.0
277,277,554,555,How do most people die?,How do people die?,0,"[people, die]","[people, die]",0,1.0
291,291,582,583,"Is there an end to the universe, and if not, i...",Is the Universe infinite or is there an end to...,1,"[end, universe, universe, infinite]","[universe, infinite, end, universe]",0,1.0
107,107,215,216,What's the difference between love and pity?,What is the difference between love and pity?,1,"[difference, love, pity]","[difference, love, pity]",0,1.0
609,609,1216,1217,How can eating prunes help during a constipation?,Does eating prunes help with constipation?,1,"[eating, prunes, constipation]","[eating, prunes, constipation]",0,1.0
601,601,1200,1201,What should I do for Web design?,What Is Web Design?,0,"[web, design]","[web, design]",0,1.0


In [54]:
dataset.sort_values(by='cos_sim', ascending=True)[0:10]

,id,qid1,qid2,question1,question2,is_duplicate,question1_token,question2_token,flag_len,cos_sim
999,999,1993,1994,What is a good song for lyric prank?,Diving the Blue Hole in Dahab?,0,"[good, song, lyric, prank]","[diving, blue, hole, dahab]",0,0.0
218,218,437,438,How do I utilize free time to avoid depression?,Can a person graduating from IIMs like LIK and...,0,"[utilize, free, time, avoid, depression]","[person, graduating, iims, lik, iims, ranchi, ...",0,0.0
783,783,1561,1562,How do the PM and Prez get women?,I worked for 1month and absconded from accentu...,0,"[pm, prez, women]","[worked, 1month, absconded, accenture, bond, p...",0,0.0
223,223,447,448,What are some examples of sentences using the ...,What is the best home wireless network setup a...,0,"[examples, sentences, word, hysteria]","[best, home, wireless, network, setup, expecte...",0,0.0
233,233,467,468,Do porn stars watch porn?,How can I book Ronda Rousey to star in an adul...,0,"[porn, stars, watch, porn]","[book, ronda, rousey, star, adult, movie]",0,0.0
235,235,471,472,Why are we worried about others' opinions?,Why do we care for others' opinion and about w...,1,"[worried, opinions]","[care, opinion]",0,0.0
767,767,1529,1530,How has the vertebral column anatomy changed t...,When calculating bullet spin; MV X 12 (twist r...,0,"[vertebral, column, anatomy, changed, time]","[calculating, bullet, spin, mv, 12, twist, rat...",0,0.0
763,763,1521,1522,Does any one have ebook of answers of wren and...,Why did barvaria join Germany?,0,"[ebook, answers, wren, martin, grammer, compos...","[barvaria, join, germany]",0,0.0
762,762,1519,1520,Jawed habib haircut prices?,DO YOU THINK IF MY TREE FALLS ON YOUR PROPERTY...,0,"[jawed, habib, haircut, prices]","[tree, falls, property, myine, responsability]",0,0.0
758,758,1511,1512,Cochin to London etihad is the change over tim...,What is difference between China and the Unite...,0,"[cochin, london, etihad, change, time, 1hr, su...","[difference, china, united, states, send, gifts]",0,0.0
